# Task
Make WordCount program for all the names in the dataset. Name is a word with the following properties:

1.The first character is not a digit (other characters can be digits). 


2.The first character is uppercase, all the other characters that are letters are lowercase.


3.There are less than 0.5% occurrences of this word, when this word regardless to its case appears in the dataset and the condition (2) is not met.

4.Order by quantity, most popular first, output format:

name <tab> count

The result is the 5th line in the output, for example:

john 1234

In [28]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode
    
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        #text = line.strip()
        #words = re.split("\W*\s+\W*", text)
    except ValueError as e:
        continue
        
    for word in words:
        #we eliminate all strings that satisfy first condition since they cannot be a name at all
        if not word.isalpha():
            continue
                
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        is_name = 1 if re.search('^[A-Z][a-z]+', word) else 0
        print "%s\t%d\t%d" % (word.lower(), 1, is_name)
        #print(str(word)+'\t'+str(1)+'\t'+str(is_name))


Overwriting mapper.py


In [29]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0
is_name_count=0

for line in sys.stdin:
    try:
        key, count, is_name = line.strip().split('\t', 2)
        count = int(count)
        is_name = int(is_name)
    except ValueError as e:
        #print(e)
        continue
        
    if current_key != key:
        if current_key:
           #print(current_key, is_name_count, word_sum)
            if (word_sum-is_name_count)/float(word_sum)*100<0.5:
                print "%s\t%d" % (current_key, word_sum)
                #print(current_key, word_sum)
        word_sum = 0
        current_key = key
        is_name_count=0
    
    is_name_count+=is_name
    word_sum += count

if current_key:
    if (count-is_name_count)/float(count)*100<0.5:
        print "%s\t%d" % (current_key, word_sum)
        #print(current_key, word_sum)


Overwriting reducer.py


In [30]:
%%writefile combiner.py

import sys

current_key = None
word_sum = 0
is_name_count=0

for line in sys.stdin:
    try:
        key, count, is_name = line.strip().split('\t', 2)
        count = int(count)
        is_name = int(is_name)
    except ValueError as e:
        #print(e)
        continue
        
    if current_key != key:
        if current_key:
           #print(current_key, is_name_count, word_sum)
           print "%s\t%d\t%d" % (current_key, word_sum, is_name_count)
                #print(current_key, word_sum)
        word_sum = 0
        current_key = key
        is_name_count=0
    
    is_name_count+=is_name
    word_sum += count

if current_key:
    print "%s\t%d\t%d" % (current_key, word_sum, is_name_count)
        #print(current_key, word_sum)


Overwriting combiner.py


In [31]:
%%writefile mapper_rating.py

import sys
reload(sys)

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
        
    print "%s\t%d" % (word, count)


Overwriting mapper_rating.py


In [32]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [34]:
%%bash

OUT_DIR="name_count_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming nameCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,combiner.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python combiner.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_DIR2="name_rating"
NUM_REDUCERS2=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming RatingNameCount" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.map.output.field.separator='\t'  \
    -D mapreduce.partition.keycomparator.options=-k2,2nr \
    -D mapreduce.job.reduces=${NUM_REDUCERS2} \
    -files mapper_rating.py \
    -mapper "python mapper_rating.py" \
    -reducer "python mapper_rating.py" \
    -input  ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null

hdfs dfs -cat name_rating/part-00000 | head -5 | tail -1

french	5753


rm: `name_count_result_1525951237802486': No such file or directory
18/05/10 11:20:41 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/10 11:20:41 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/10 11:20:42 INFO mapred.FileInputFormat: Total input files to process : 1
18/05/10 11:20:42 INFO mapreduce.JobSubmitter: number of splits:2
18/05/10 11:20:42 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1525948943003_0012
18/05/10 11:20:42 INFO impl.YarnClientImpl: Submitted application application_1525948943003_0012
18/05/10 11:20:42 INFO mapreduce.Job: The url to track the job: http://73035a96f92d:8088/proxy/application_1525948943003_0012/
18/05/10 11:20:42 INFO mapreduce.Job: Running job: job_1525948943003_0012
18/05/10 11:20:48 INFO mapreduce.Job: Job job_1525948943003_0012 running in uber mode : false
18/05/10 11:20:48 INFO mapreduce.Job:  map 0% reduce 0%
18/05/10 11:21:05 INFO mapreduce.Job:  map 18% reduce 0%
18/05/10 11: